In [ ]:
import sys, os, subprocess
from bottle import route, run, static_file, ServerAdapter
from gevent.pywsgi import WSGIServer

class SSLWebServer(ServerAdapter):
    def run(self, handler):
        srv = WSGIServer( (self.host, self.port), handler,
        certfile='./server.pem', keyfile='./server.pem')
        srv.serve_forever()

@route( '/<lon:float>/<lat:float>/' )
def index(lon, lat):
    texPath = '/dir/path/texture000.png'
    if( os.path.exists( texPath ) ):
        os.remove(texPath)
    scrText = f'''StelMovementMgr.zoomTo(180) ;
core.setObserverLocation({lon},{lat},0);
StelMovementMgr.lookEast(); core.wait(0.0);
core.screenshot("texture"); core.quitStellarium ();'''
    stellariumPath = '/Applications/Stellarium.app/Contents/'
    path = stellariumPath + 'Resources/scripts/screencapture.ssc'
    with open(path, 'w') as f:
        f.write( scrText )
    cmd = stellariumPath +'/MacOS/stellarium --screenshot-dir /dir/path --startup-script screencapture.ssc --full-screen no'
    
    popen = subprocess.Popen( cmd.strip().split(" "), shell=False )
    popen.wait(); return static_file( 'index.htm', root="/dir/path")

@route('/<filename:path>')
def static(filename):
    return static_file(filename, root="/dir/path")

run(host='0.0.0.0', port=8080, server=SSLWebServer)